# XGBoost Grocery Sales Forecasting - Simplified Version

Memory-efficient training with essential features only

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error
import pickle
import warnings
import gc
from pathlib import Path
import json

warnings.filterwarnings('ignore')
print("✓ Libraries imported")

✓ Libraries imported


In [2]:
# Configuration
DATA_DIR = Path("../data")
RAW_DATA_DIR = DATA_DIR / "raw"
RESULTS_DIR = Path("../results")
MODELS_DIR = RESULTS_DIR / "models"
RESULTS_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)

# Train only first 3 horizons for demonstration
FORECAST_HORIZONS = [1, 7, 14]  
VALIDATION_DATE = "2017-07-01"

# Improved XGBoost parameters for better accuracy
XGBOOST_PARAMS = {
    'objective': 'reg:squaredlogerror',
    'eval_metric': 'rmsle',
    'learning_rate': 0.05,  # Lower learning rate for better generalization
    'max_depth': 8,  # Deeper trees to capture more patterns
    'min_child_weight': 3,  # Regularization
    'subsample': 0.8,  # Row sampling
    'colsample_bytree': 0.8,  # Column sampling
    'gamma': 0.1,  # Minimum loss reduction
    'reg_alpha': 0.1,  # L1 regularization
    'reg_lambda': 1.0,  # L2 regularization
    'n_estimators': 300,  # More trees for better accuracy
    'random_state': 42,
    'n_jobs': -1,
}

print("✓ Configuration set")


✓ Configuration set


In [3]:
# Load data - stratified sample
print("Loading data...")
train_data = pd.read_csv(DATA_DIR / 'df_train_stratified.csv', parse_dates=['date'])
print(f"Full shape: {train_data.shape}")
print(f"Date range: {train_data['date'].min()} to {train_data['date'].max()}")

# Keep only recent data (from 2016 onwards) for better validation and enough horizon coverage
train_data = train_data[train_data['date'] >= '2016-01-01'].reset_index(drop=True)
print(f"After filtering (2016+): {train_data.shape}")
print(f"Date range: {train_data['date'].min()} to {train_data['date'].max()}")
display(train_data.head())


Loading data...
Full shape: (1254984, 5)
Date range: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
After filtering (2016+): (590386, 5)
Date range: 2016-01-01 00:00:00 to 2017-08-15 00:00:00
Full shape: (1254984, 5)
Date range: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
After filtering (2016+): (590386, 5)
Date range: 2016-01-01 00:00:00 to 2017-08-15 00:00:00


,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2016-01-01,25,841842,2.484907,False
1,2016-01-01,25,1463825,3.367296,False
2,2016-01-01,25,253145,2.397895,False
3,2016-01-01,25,265258,2.302585,False
4,2016-01-01,25,517615,0.693147,False


In [4]:
# Merge supplementary data
items = pd.read_parquet(RAW_DATA_DIR / 'items.parquet')
stores = pd.read_parquet(RAW_DATA_DIR / 'stores.parquet')

items = items.reset_index()  # item_nbr is in the index
stores = stores.reset_index()  # store_nbr is in the index

# Load additional data for better features
try:
    oil = pd.read_parquet(DATA_DIR / 'df_oil_cleaned.parquet')
    oil['date'] = pd.to_datetime(oil['date'])
    print(f"✓ Loaded oil data: {oil.shape}")
except:
    oil = None
    print("⚠ Oil data not available")

# Merge
df = train_data.merge(items[['item_nbr', 'family', 'perishable']], on='item_nbr', how='left')
df = df.merge(stores[['store_nbr', 'type', 'cluster']], on='store_nbr', how='left')

# Merge oil prices if available
if oil is not None:
    df = df.merge(oil[['date', 'dcoilwtico']], on='date', how='left')
    df['dcoilwtico'] = df['dcoilwtico'].fillna(method='ffill').fillna(0)

print(f"After merge: {df.shape}")
print(f"Columns: {df.columns.tolist()}")


✓ Loaded oil data: (1218, 2)
After merge: (590386, 10)
Columns: ['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'perishable', 'type', 'cluster', 'dcoilwtico']
After merge: (590386, 10)
Columns: ['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'perishable', 'type', 'cluster', 'dcoilwtico']


In [5]:
# Create advanced features
print("Creating features...")

# Temporal features
df['dayofweek'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['day'] = df['date'].dt.day
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
df['is_month_start'] = (df['day'] <= 7).astype(int)
df['is_month_end'] = (df['day'] >= 23).astype(int)

# Encode categoricals
df['family_enc'] = df['family'].astype('category').cat.codes
df['type_enc'] = df['type'].astype('category').cat.codes

# Onpromotion
df['onpromotion'] = df['onpromotion'].astype(int)

# Sort for lag features
df = df.sort_values(['store_nbr', 'item_nbr', 'date']).reset_index(drop=True)

# Lag features (previous sales) - these are safe, using past data only
print("Creating lag features...")
for lag in [1, 7, 14]:
    df[f'lag_{lag}'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(lag)

# Rolling statistics - using shift to ensure only past data
print("Creating rolling features...")
for window in [7, 14]:
    df[f'rolling_mean_{window}'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).mean()
    )
    df[f'rolling_std_{window}'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).std()
    ).fillna(0)

print("✓ Features created (target encodings will be added after train/val split)")


Creating features...
Creating lag features...
Creating rolling features...
Creating lag features...
Creating rolling features...
✓ Features created (target encodings will be added after train/val split)
✓ Features created (target encodings will be added after train/val split)


In [6]:
# Define feature columns - now with advanced features
feature_cols = [
    # Basic features
    'store_nbr', 'item_nbr', 'onpromotion',
    'cluster', 'perishable',
    # Temporal features
    'dayofweek', 'month', 'year', 'day', 'is_weekend', 'is_month_start', 'is_month_end',
    # Categorical encodings
    'family_enc', 'type_enc',
    # Lag features
    'lag_1', 'lag_7', 'lag_14',
    # Rolling features
    'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14',
    # Target encodings
    'store_mean_sales', 'item_mean_sales', 'family_mean_sales',
]

# Add oil price if available
if 'dcoilwtico' in df.columns:
    feature_cols.append('dcoilwtico')

print(f"Features: {feature_cols}")
print(f"Total: {len(feature_cols)}")


Features: ['store_nbr', 'item_nbr', 'onpromotion', 'cluster', 'perishable', 'dayofweek', 'month', 'year', 'day', 'is_weekend', 'is_month_start', 'is_month_end', 'family_enc', 'type_enc', 'lag_1', 'lag_7', 'lag_14', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 'rolling_std_14', 'store_mean_sales', 'item_mean_sales', 'family_mean_sales', 'dcoilwtico']
Total: 25


In [7]:
# Train/val split
val_date = pd.to_datetime(VALIDATION_DATE)
train_df = df[df['date'] < val_date].copy()
val_df = df[df['date'] >= val_date].copy()

print(f"Train: {train_df.shape} ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"Val: {val_df.shape} ({val_df['date'].min()} to {val_df['date'].max()})")

# IMPORTANT: Add target encodings ONLY using training data to prevent leakage
print("\nCreating target encodings from training data only...")
store_means = train_df.groupby('store_nbr')['unit_sales'].mean()
item_means = train_df.groupby('item_nbr')['unit_sales'].mean()
family_means = train_df.groupby('family')['unit_sales'].mean()

# Global mean for unseen categories
global_mean = train_df['unit_sales'].mean()

# Apply to train set
train_df['store_mean_sales'] = train_df['store_nbr'].map(store_means).fillna(global_mean)
train_df['item_mean_sales'] = train_df['item_nbr'].map(item_means).fillna(global_mean)
train_df['family_mean_sales'] = train_df['family'].map(family_means).fillna(global_mean)

# Apply to validation set (using training statistics)
val_df['store_mean_sales'] = val_df['store_nbr'].map(store_means).fillna(global_mean)
val_df['item_mean_sales'] = val_df['item_nbr'].map(item_means).fillna(global_mean)
val_df['family_mean_sales'] = val_df['family'].map(family_means).fillna(global_mean)

print("✓ Target encodings added without leakage")

del df
gc.collect()
print("✓ Split complete")


Train: (541842, 26) (2016-01-01 00:00:00 to 2017-06-30 00:00:00)
Val: (48544, 26) (2017-07-01 00:00:00 to 2017-08-15 00:00:00)

Creating target encodings from training data only...
✓ Target encodings added without leakage
✓ Split complete
✓ Split complete


In [8]:
# Define metrics
def rmsle(y_true, y_pred):
    y_true = np.maximum(y_true, 0)
    y_pred = np.maximum(y_pred, 0)
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

print("✓ Metrics defined")

✓ Metrics defined


In [9]:
# Train models
models = {}
results = []

for horizon in FORECAST_HORIZONS:
    print(f"\n{'='*60}")
    print(f"Training Horizon {horizon}")
    print(f"{'='*60}")
    
    # Create target
    train_h = train_df.copy()
    val_h = val_df.copy()
    
    train_h = train_h.sort_values(['store_nbr', 'item_nbr', 'date'])
    val_h = val_h.sort_values(['store_nbr', 'item_nbr', 'date'])
    
    train_h['target'] = train_h.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(-horizon)
    val_h['target'] = val_h.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(-horizon)
    
    train_h = train_h.dropna(subset=['target'])
    val_h = val_h.dropna(subset=['target'])
    
    X_train = train_h[feature_cols].fillna(0)
    y_train = train_h['target'].values
    
    X_val = val_h[feature_cols].fillna(0)
    y_val = val_h['target'].values
    
    print(f"Train: {X_train.shape[0]:,} samples")
    print(f"Val: {X_val.shape[0]:,} samples")
    
    # Skip if not enough data
    if X_train.shape[0] < 10 or X_val.shape[0] < 10:
        print(f"⚠ Skipping horizon {horizon} - insufficient data")
        continue
    
    # Train
    model = xgb.XGBRegressor(**XGBOOST_PARAMS)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # Predict
    y_pred_train = np.maximum(model.predict(X_train), 0)
    y_pred_val = np.maximum(model.predict(X_val), 0)
    
    # Metrics
    train_rmsle = rmsle(y_train, y_pred_train)
    val_rmsle = rmsle(y_val, y_pred_val)
    
    # Calculate approximate accuracy
    train_accuracy = 1 / (1 + train_rmsle) * 100
    val_accuracy = 1 / (1 + val_rmsle) * 100
    
    print(f"\nTrain RMSLE: {train_rmsle:.6f} (Accuracy: {train_accuracy:.2f}%)")
    print(f"Val RMSLE: {val_rmsle:.6f} (Accuracy: {val_accuracy:.2f}%)")
    
    models[f'h{horizon}'] = model
    results.append({
        'horizon': horizon,
        'train_rmsle': train_rmsle,
        'val_rmsle': val_rmsle,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy
    })
    
    del train_h, val_h, X_train, X_val
    gc.collect()

print("\n" + "="*60)
print(f"✓ TRAINED {len(models)}/{len(FORECAST_HORIZONS)} MODELS")
print("="*60)



Training Horizon 1
Train: 398,537 samples
Val: 7,794 samples
Train: 398,537 samples
Val: 7,794 samples

Train RMSLE: 0.219205 (Accuracy: 82.02%)
Val RMSLE: 0.222759 (Accuracy: 81.78%)

Training Horizon 7
Train: 18,556 samples
Val: 0 samples
⚠ Skipping horizon 7 - insufficient data

Training Horizon 14

Train RMSLE: 0.219205 (Accuracy: 82.02%)
Val RMSLE: 0.222759 (Accuracy: 81.78%)

Training Horizon 7
Train: 18,556 samples
Val: 0 samples
⚠ Skipping horizon 7 - insufficient data

Training Horizon 14
Train: 38 samples
Val: 0 samples
⚠ Skipping horizon 14 - insufficient data

✓ TRAINED 1/3 MODELS
Train: 38 samples
Val: 0 samples
⚠ Skipping horizon 14 - insufficient data

✓ TRAINED 1/3 MODELS


In [ ]:
# Results summary
results_df = pd.DataFrame(results)
print("\nRESULTS:")
print(results_df.to_string(index=False))

print(f"\nAverage Val RMSLE: {results_df['val_rmsle'].mean():.6f}")
print(f"Average Val Accuracy: {results_df['val_accuracy'].mean():.2f}%")

print(f"\n{'='*60}")
print("COMPARISON:")
print(f"{'='*60}")
print(f"Baseline Model (simple):  RMSLE 0.280 ≈ 78% accuracy")
print(f"Current Model (improved): RMSLE {results_df['val_rmsle'].mean():.3f} ≈ {results_df['val_accuracy'].mean():.1f}% accuracy")
print(f"Improvement: +{results_df['val_accuracy'].mean() - 78:.1f}% accuracy gain")
print(f"{'='*60}")


RESULTS:
 horizon  train_rmsle  val_rmsle  train_accuracy  val_accuracy
       1     0.219205   0.222759       82.020669     81.782256

Average Val RMSLE: 0.222759
Average Val Accuracy: 81.78%

COMPARISON:
Baseline Model (simple):  RMSLE 0.280 ≈ 78% accuracy
Current Model (improved): RMSLE 0.223 ≈ 81.8% accuracy
Improvement: +3.8% accuracy gain


In [11]:
# Save models
print("Saving models...")

for key, model in models.items():
    path = MODELS_DIR / f"xgboost_{key}_simple.pkl"
    with open(path, 'wb') as f:
        pickle.dump(model, f)
    print(f"  ✓ Saved {path.name}")

# Save results
results_df.to_csv(RESULTS_DIR / "training_results_simple.csv", index=False)
print(f"  ✓ Saved results")

# Save feature list
with open(MODELS_DIR / "features_simple.json", 'w') as f:
    json.dump(feature_cols, f, indent=2)
print(f"  ✓ Saved features")

print("\n✓ ALL DONE!")

Saving models...
  ✓ Saved xgboost_h1_simple.pkl
  ✓ Saved results
  ✓ Saved features

✓ ALL DONE!
